In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_column',None)

In [2]:
folder2 = '~/Repos/TFM-Test-your-business-viability/Data/censolocales/'

In [3]:
cd {folder2}

/home/dsc/Repos/TFM-Test-your-business-viability/Data/censolocales


In [4]:
pwd

'/home/dsc/Repos/TFM-Test-your-business-viability/Data/censolocales'

In [5]:
df_epi19 = pd.read_csv('OPEN DATA Locales-Epigrafes201908_clean.csv',sep=';',encoding='latin9')
df_epi18 = pd.read_csv('OPEN DATA Locales-Epigrafes201808_clean.csv',sep=';',encoding='latin9')

/home/dsc/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df_epi19['conc'] = df_epi19.rotulo.str.strip() + "-" + df_epi19.desc_vial_acceso.str.strip() + "-" + df_epi19.num_acceso.astype(str) 
df_epi18['conc'] = df_epi18.rotulo.str.strip() + "-" + df_epi18.desc_vial_acceso.str.strip() + "-" + df_epi18.num_acceso.astype(str)    

In [7]:
df_epi19['id_local_norm'] = df_epi19['id_local']
df_epi18['id_local_norm'] = df_epi18['id_local']

In [48]:
df_epi19_sa = df_epi19[df_epi19.desc_tipo_acceso_local != 'Agrupado']
df_epi18_sa = df_epi18[df_epi18.desc_tipo_acceso_local != 'Agrupado']

In [21]:
df = df_epi19_sa.groupby('id_local').count().conc
df[df.values > 1]

id_local
10000003     2
10000071     2
10000102     2
10000258     4
10000306     2
10000307     2
10000308     2
10000310     2
10000398     2
10000401     3
10000422     2
10000434     4
10000441     2
10000463     2
10000502     2
10000684     2
10000708     2
10000744     3
10000854     2
10000877     2
10000960     2
10000966     2
10001020     2
10001064     2
10001141     2
10001180     2
10001207     2
10001246     2
10001299     2
10001447     2
            ..
290000552    2
290000557    2
290000560    2
290000592    2
290000603    3
290000605    2
290000610    4
290000612    2
290000620    3
290000624    4
290000625    2
290000653    2
290000655    2
290000665    3
290000686    2
290000689    2
290000692    2
290000699    2
290000711    2
290000730    2
290000743    2
290000765    2
290000789    3
290000796    2
290000799    2
290000816    2
290000817    2
290000823    3
290000824    2
290000880    2
Name: conc, Length: 10671, dtype: int64

In [49]:
df_epi19_a = df_epi19[df_epi19.desc_tipo_acceso_local == 'Agrupado']
df_epi18_a = df_epi18[df_epi18.desc_tipo_acceso_local == 'Agrupado']

In [11]:
arr1 = df_epi19_sa.id_local.unique()
arr2 = df_epi18_sa.id_local.unique()
missing_id_df_epi19 = [x for x in arr1 if x not in arr2]
missing_id_df_epi18 = [x for x in arr2 if x not in arr1]
    
arr3 = df_epi19_sa[df_epi19_sa.id_local.isin(missing_id_df_epi19)].conc.unique()
arr4 = df_epi18_sa.conc.unique()
equal_conc_epi19 = [x for x in arr3 if x in arr4]
equal_conc_epi18 = [x for x in arr4 if x in arr3]
len(missing_id_df_epi19), len(missing_id_df_epi18),len(equal_conc_epi19),len(equal_conc_epi18)

(534, 21, 45, 45)

In [14]:
len(df_epi19_sa), len(df_epi18_sa), len(df_epi19_a), len(df_epi18_a)


(147776, 147109, 15540, 15234)

Función que identifica diferencias de locales entre un df y otro y que cambia el id local si el rótulo+dir+num son iguales en ambos ficheros

In [15]:
def missing_id(df1,df2):
    arr1 = []
    arr2 = []
    missing_id_df1 = []
    missing_id_df2 = []
    
    df1.drop_duplicates()
    df2.drop_duplicates()
    
    arr1 = df1.id_local.unique()
    arr2 = df2.id_local.unique()
    missing_id_df1 = [x for x in arr1 if x not in arr2]
    missing_id_df2 = [x for x in arr2 if x not in arr1]
    
    return(missing_id_df1,missing_id_df2)

In [16]:
def id_situacion(df):
    df.loc[df['desc_situacion_local'] == 'Abierto','id_situacion_local'] = 1
    df.loc[df['desc_situacion_local'] == 'Cerrado','id_situacion_local'] = 4
    df.loc[df['desc_situacion_local'] == 'Uso vivienda','id_situacion_local'] = 5
    df.loc[df['desc_situacion_local'] == 'En obras','id_situacion_local'] = 7
    df.loc[df['desc_situacion_local'] == 'Baja','id_situacion_local'] = 8
    df.loc[df['desc_situacion_local'] == 'Baja Reunificacion','id_situacion_local'] = 9
    df.loc[df['desc_situacion_local'] == 'Baja PC Asociado','id_situacion_local'] = 10
    return df

In [17]:
def estado(column1, column2):
    if len(column1) != len(column2):
        return 'Error'
    conditions = [
        (column1 == 'LOCAL SIN ACTIVIDAD') & (column2 == 'Abierto'), 
        (column2 == 'Baja'),
        (column2 == 'Baja PC Asociado'),
        (column2 == 'Baja Reunificacion'),
        (column1 == 'LOCAL SIN ACTIVIDAD') & (column2 == 'En obras'),
        (column2 == 'Cerrado'),
        (column2 == 'Uso vivienda'),
        (column2 == 'Abierto'),
        (column2 == 'En obras')
        ]
    outputs = ['Cerrado','Cerrado','Cerrado','Cerrado','Cerrado',
               'Cerrado','Uso vivienda','Abierto','En obras']
    res = np.select(conditions,outputs,'Other')
    
    return res

In [18]:
def clear_id(df1, df2): 
    
    arr1 = []
    arr2 = []
    arr3 = []
    arr4 = []
    equal_conc_1 = []
    equal_conc_2 = []
    
    df1.drop_duplicates()
    df2.drop_duplicates()
    
    df2['conc'] = df2.rotulo.str.strip() + "-" + df2.desc_vial_acceso.str.strip() + "-" + df2.num_acceso.astype(str) 
    df1['conc'] = df1.rotulo.str.strip() + "-" + df1.desc_vial_acceso.str.strip() + "-" + df1.num_acceso.astype(str) 
    
    missing_id_df1_, missing_id_df2_ = missing_id(df1,df2)
    
    arr1 = df1[df1.id_local.isin(missing_id_df1_)].conc.unique()
    arr2 = df2.conc.unique()
    equal_conc_1 = [x for x in arr1 if x in arr2]
    
    df1_ = df1.copy()
    df2_ = df2.copy()
    
    for i in equal_conc_1:
        df1_.loc[df1_.conc == i,'id_local'] = min(df1_.loc[df1_.conc == i,'id_local'].values)
        df2_.loc[df2_.conc == i,'id_local'] = df1_.loc[df1_.conc == i,'id_local'].values[0]
    
    df1_.drop_duplicates(subset=['id_local','conc','desc_epigrafe'],keep='last',inplace=True)
    df2_.drop_duplicates(subset=['id_local','conc','desc_epigrafe'],keep='last',inplace=True)
    
    missing_id_df1_, missing_id_df2_ = missing_id(df1_,df2_)

    arr3 = df2_[df2_.id_local.isin(missing_id_df2_)].conc.unique()
    arr4 = df1_.conc.unique()
    
    equal_conc_2 = [x for x in arr3 if x in arr4]
    
    for i in equal_conc_2:
        df2_.loc[df2_.conc == i,'id_local'] = max(df2_.loc[df2_.conc == i,'id_local'].values)
        df1_.loc[df1_.conc == i,'id_local'] = df2_.loc[df2_.conc == i,'id_local'].values[0]
    
    df1_.drop_duplicates(subset=['id_local','conc','desc_epigrafe'],keep='last',inplace=True)
    df2_.drop_duplicates(subset=['id_local','conc','desc_epigrafe'],keep='last',inplace=True)
    
    # completo el fichero final con los ids missing del fichero del año anterior
    missing_id_df1_, missing_id_df2_ = missing_id(df1_,df2_)
    if len(missing_id_df2_) > 0:
        df_concat = df2_[df2_.id_local.isin(missing_id_df2_)]
        df_concat.desc_situacion_local = 'Cerrado'
    
    df1_final = pd.concat([df1_,df_concat])
    
    #completo el 'id_situacion_local' vacio
    df1_final_ = id_situacion(df1_final)
    
    #trato los NaN
    df1_final_['id_seccion'].fillna('Z',inplace=True)
    df1_final_['desc_seccion'].fillna('SIN ACTIVIDAD',inplace=True)
    df1_final_['id_division'].fillna(0,inplace=True)
    df1_final_['desc_division'].fillna('SIN ACTIVIDAD',inplace=True)
    df1_final_['id_epigrafe'].fillna(0,inplace=True)
    df1_final_['desc_epigrafe'].fillna('LOCAL SIN ACTIVIDAD',inplace=True)
    df1_final_['nombre_agrupacion'].fillna('SIN AGRUPACION',inplace=True)
    df1_final_['desc_tipo_agrup'].fillna('SIN AGRUPACION',inplace=True)
    df1_final_['id_tipo_agrup'].fillna(-1,inplace=True)

    #unifico estados
    df1_final_['desc_sit_loc_modif'] = estado(df1_final_['desc_epigrafe'], df1_final_['desc_situacion_local'])

    
    return(df1_final_,df2_)

In [19]:
df_epi19_,df_epi18_ = clear_id(df_epi19_sa,df_epi18_sa)

/home/dsc/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/dsc/.local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/dsc/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

In [20]:
arr1 = df_epi19_.id_local.unique()
arr2 = df_epi18_.id_local.unique()
missing_id_df_epi19_ = [x for x in arr1 if x not in arr2]
missing_id_df_epi18_ = [x for x in arr2 if x not in arr1]
    
arr3 = df_epi19_[df_epi19_.id_local.isin(missing_id_df_epi19_)].conc.unique()
arr4 = df_epi18_.conc.unique()
equal_conc_epi19_ = [x for x in arr3 if x in arr4]
equal_conc_epi18_ = [x for x in arr4 if x in arr3]
len(missing_id_df_epi19_), len(missing_id_df_epi18_),len(equal_conc_epi19_),len(equal_conc_epi18_)

(493, 0, 0, 0)

In [16]:
equal_conc_epi19_

['HEDONAI-CALDERILLA-1',
 'PESCADERIA-ALBERTO PALACIOS-18',
 'PESCADERIA-BRAVO MURILLO-122',
 'ROTULO NO INFORMADO-ERMITA DEL SANTO-48',
 'ROTULO NO INFORMADO-LUIS ARAGONES-4',
 'LA RAPA-CALDERILLA-1',
 'CENTROS UNICO-CALDERILLA-1']

In [139]:
df_epi19[df_epi19.conc == 'MULTIOPTICAS-CALDERILLA-1']

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc
151861,280052463,11,CARABANCHEL,1106,BUENAVISTA,NaN,NaN,198,"437367,57","4468536,54",NaN,Agrupado,NaN,Abierto,31000314,CALLE,CALDERILLA ...,31025055,1,NUM,1,,0,NaN,CALLE,CALDERILLA ...,31025055,1,NUM,1,,"437367,57","4468536,54",99000252.0,CENTRO COMERCIAL ISLAZUL,1.0,Centro Comercial,PB,67E,MULTIOPTICAS,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",477806,COMERCIO AL POR MENOR DE MATERIAL DE OPTICA,MULTIOPTICAS-CALDERILLA-1


In [138]:
len(df_epi19[df_epi19.desc_tipo_acceso_local == 'Agrupado'])

15540

In [140]:
len(df_epi18[df_epi18.desc_tipo_acceso_local == 'Agrupado'])

15234

In [17]:
columns_to_merge = ['conc','desc_epigrafe']
columns_to_drop =  ['id_local','id_distrito_local', 'desc_distrito_local',
       'id_barrio_local', 'desc_barrio_local', 'cod_barrio_local',
       'id_seccion_censal_local', 'desc_seccion_censal_local',
       'coordenada_x_local', 'coordenada_y_local', 'id_tipo_acceso_local',
       'desc_tipo_acceso_local', 'id_situacion_local', 'id_vial_edificio', 'clase_vial_edificio', 
        'desc_vial_edificio',
       'id_ndp_edificio', 'id_clase_ndp_edificio', 'nom_edificio',
       'num_edificio', 'cal_edificio', 'secuencial_local_PC', 'id_vial_acceso',
       'clase_vial_acceso', 'desc_vial_acceso', 'id_ndp_acceso',
       'id_clase_ndp_acceso', 'nom_acceso', 'num_acceso', 'cal_acceso',
       'coordenada_x_agrupacion', 'coordenada_y_agrup', 'id_agrupacion',
       'nombre_agrupacion', 'id_tipo_agrup', 'desc_tipo_agrup',
       'id_planta_agrupado', 'id_local_agrupado', 'rotulo', 'id_seccion',
       'desc_seccion', 'id_division', 'desc_division', 'id_epigrafe']

In [18]:
df_epi18_d = df_epi18_.drop(columns = columns_to_drop)

In [19]:
df_epi_final = df_epi19_.merge(df_epi18_d, how='left', on=['conc','desc_epigrafe'],
         suffixes=('_x', '_y'))

In [20]:
len(df_epi19_), len(df_epi_final)

(163017, 165274)

In [21]:
df_epi_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165274 entries, 0 to 165273
Data columns (total 49 columns):
id_local                     165274 non-null int64
id_distrito_local            165274 non-null int64
desc_distrito_local          165274 non-null object
id_barrio_local              165274 non-null int64
desc_barrio_local            165274 non-null object
cod_barrio_local             0 non-null float64
id_seccion_censal_local      0 non-null float64
desc_seccion_censal_local    165274 non-null int64
coordenada_x_local           165274 non-null object
coordenada_y_local           165274 non-null object
id_tipo_acceso_local         0 non-null float64
desc_tipo_acceso_local       165274 non-null object
id_situacion_local           165274 non-null float64
desc_situacion_local_x       165274 non-null object
id_vial_edificio             165274 non-null int64
clase_vial_edificio          165274 non-null object
desc_vial_edificio           165274 non-null object
id_ndp_edificio      

In [22]:
df_epi_final[df_epi_final.desc_situacion_local_y.isnull() & 
             (df_epi_final.desc_epigrafe != 'LOCAL SIN ACTIVIDAD')].groupby(['desc_sit_loc_modif','desc_distrito_local','desc_epigrafe']).count() 

id_local  \
desc_sit_loc_modif desc_distrito_local  desc_epigrafe                                                  
Abierto            ARGANZUELA           ACABADO DE EDIFICIOS (CARPINTERIA, REVOCAMIENTO...         1   
                                        ACTIVIDADES ADMINISTRATIVAS Y AUXILIARES DE OFI...        22   
                                        ACTIVIDADES DE APOYO A LAS EMPRESAS N.C.O.P. (A...         1   
                                        ACTIVIDADES DE BIBLIOTECAS, ARCHIVOS, MUSEOS Y ...         2   
                                        ACTIVIDADES DE CONSULTORIA DE GESTION EMPRESARIAL          1   
                                        ACTIVIDADES DE CREACION, ARTISTICAS Y ESPECTACULOS         1   
                                        ACTIVIDADES DE LAS AGENCIAS DE COLOCACION                  1   
                                        ACTIVIDADES DE LAS SEDES CENTRALES                         1   
                                        ACTIVIDADES DE LOS CENTROS DE LLAMADAS                     1   
                                        ACTIVIDADES DE LOS GIMNASIOS                              11   
                                        ACTIVIDADES DE ORGANIZACIONES RELIGIOSAS                   1   
                                        ACTIVIDADES DE PROGRAMACION Y EMISION DE RADIO ...         1   
                                        ACTIVIDADES DE SERVICIOS SOCIALES SIN ALOJAMIEN...         4   
                                        ACTIVIDADES INMOBILIARIAS (COMPRAVENTA, ALQUILE...         8   
                                        ADIESTRAMIENTO, CUIDADO DE ANIMALES, RESIDENCIA...         1   
                                        ADMINISTRACION PUBLICA Y DEFENSA. SEGURIDAD SOC...         1   
                                        ALBERGUES JUVENILES Y OTROS ALOJAMIENTOS TURIST...         1   
                                        ALMACEN DE DISTRIBUCION DE PRODUCTOS ALIMENTICIOS          1   
                                        ALQUILER DE BIENES INMOBILIARIOS POR CUENTA PROPIA         1   
                                        APARCAMIENTOS PRIVADOS                                     1   
                                        ARREGLO DE ROPA                                            1   
                                        ASISTENCIA EN ESTABLECIMIENTOS RESIDENCIALES PA...         1   
                                        AUTOESCUELA                                                2   
                                        BAR CON COCINA                                            21   
                                        BAR ESPECIAL SIN ACTUACIONES                               3   
                                        BAR RESTAURANTE                                           25   
                                        BAR SIN COCINA                                             5   
                                        CAFETERIA                                                 15   
                                        CASAS DE HUESPEDES                                         1   
                                        CENTRO DE ESTETICA                                        20   
...                                                                                              ...   
Cerrado            VICALVARO            BAR CON COCINA                                             1   
                                        FABRICACION DE APARATOS ELECTRICOS, ELECTRODOME...         1   
                                        PROMOCION INMOBILIARIA                                     1   
                   VILLA DE VALLECAS    ALMACEN DE DISTRIBUCION A TEMPERATURA DE CONGEL...         1   
                                        ALMACEN DE DISTRIBUCION A TEMPERATURA DE REFRIG...         1   
                                        ALMACEN DE DISTRIBUCION DE PRODUCTOS ALIMENTICIOS          1   
                                    

In [23]:
df_epi_final[df_epi_final.conc == 'HEDONAI-CALDERILLA-1']

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local_x,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,desc_sit_loc_modif,desc_situacion_local_y
77640,270564453,11,CARABANCHEL,1106,BUENAVISTA,NaN,NaN,198,0,0,NaN,Agrupado,1.0,Abierto,31000314,CALLE,CALDERILLA ...,31025055,1,NUM,1,,0,NaN,CALLE,CALDERILLA ...,31025055,1,NUM,1,,"437367,57","4468536,54",99000252.0,CENTRO COMERCIAL ISLAZUL,1.0,Centro Comercial,PB,03,HEDONAI,S,OTROS SERVICIOS,96,OTROS SERVICIOS PERSONALES,960203,CENTRO DE ESTETICA,HEDONAI-CALDERILLA-1,Abierto,Abierto
129528,110003842,11,CARABANCHEL,1106,BUENAVISTA,NaN,NaN,198,"437367,57","4468536,54",NaN,Agrupado,9.0,Baja Reunificacion,31000314,CALLE,CALDERILLA ...,31025055,1,NUM,1,,0,NaN,CALLE,CALDERILLA ...,31025055,1,NUM,1,,"437367,57","4468536,54",99000252.0,CENTRO COMERCIAL ISLAZUL,1.0,Centro Comercial,SO,3A,HEDONAI,S,OTROS SERVICIOS,96,OTROS SERVICIOS PERSONALES,960203,CENTRO DE ESTETICA,HEDONAI-CALDERILLA-1,Cerrado,Abierto


# iterating over two columns, use `zip`
result = [f(x, y) for x, y in zip(df['col1'], df['col2'])]
# iterating over multiple columns
result = [f(row[0], ..., row[n]) for row in df[['col1', ...,'coln']]

In [37]:
id_agrupa_un_19 = df_epi19_a.id_agrupacion.unique()

In [45]:
len(df_epi19_a[df_epi19_a.id_agrupacion.isin(id_agrupa_un_19)].conc.unique())

10460

In [46]:
conc_un_a = df_epi19_a[df_epi19_a.id_agrupacion.isin(id_agrupa_un_19)].conc.unique()
len(conc_un_a)

10460

In [87]:
len(df_epi19_a.conc.values)

15540

In [122]:
len(df_epi18_a.conc.values)

15234

In [149]:
df_epi19_a.reset_index(drop=True,inplace=True)

In [150]:
df_epi18_a.reset_index(drop=True,inplace=True)

In [171]:
df_epi18_a.sort_values(['id_local'],inplace=True)
df_epi18_a.reset_index(drop=True,inplace=True)

/home/dsc/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [161]:
df_epi19_a.sort_values(['id_local'],inplace=True)

/home/dsc/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [172]:
df_epi19_a.head()

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,id_local_norm,id_local_norm.values
0,10000116,1,CENTRO,105,UNIVERSIDAD,NaN,NaN,112,0,0,NaN,Agrupado,NaN,Abierto,524500,PLAZA,MOSTENSES ...,11006826,1,NUM,1,,0,NaN,PLAZA,MOSTENSES ...,11006826,1,NUM,1,,"439852,6","4474996,53",99000061.0,MERCADO MUNICIPAL LOS MOSTENSES,12.0,Mercado Municipal,1,17,PERFUMERIA DROGUERIA RICARDO / LOCAL 17 AL 22,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",477501,COMERCIO AL POR MENOR DE ARTICULOS DE PERFUMER...,PERFUMERIA DROGUERIA RICARDO / LOCAL 17 AL 22-...,290001655.0,10000116.0
1,10000162,1,CENTRO,104,JUSTICIA,NaN,NaN,88,0,0,NaN,Agrupado,NaN,Abierto,92900,CALLE,BARCELO ...,11005232,1,NUM,6,,0,NaN,CALLE,BARCELO ...,11005232,1,NUM,6,,"440692,59","4475369,53",99000056.0,MERCADO MUNICIPAL DE BARCELO,12.0,Mercado Municipal,0,101,POLLERIAS RODRIGUEZ & MARTINEZ,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",472206,"COMERCIO AL POR MENOR DE AVES, HUEVOS Y CAZA S...",POLLERIAS RODRIGUEZ & MARTINEZ-BARCELO-6,290001655.0,10000162.0
2,10000224,1,CENTRO,102,EMBAJADORES,NaN,NaN,35,0,0,NaN,Agrupado,NaN,Abierto,262000,CALLE,EMBAJADORES ...,11002143,1,NUM,41,,0,NaN,CALLE,EMBAJADORES ...,11002143,1,NUM,41,,"440295,59","4473246,53",99000057.0,MERCADO MUNICIPAL DE SAN FERNANDO,12.0,Mercado Municipal,PB,70,SIN ROTULO,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",472201,COMERCIO AL POR MENOR DE CARNICERIA,SIN ROTULO-EMBAJADORES-41,290001655.0,NaN
3,10000226,1,CENTRO,102,EMBAJADORES,NaN,NaN,59,0,0,NaN,Agrupado,NaN,Abierto,253200,CALLE,DUQUE DE FERNAN NUÑEZ ...,20004950,1,NUM,4,,0,NaN,CALLE,DUQUE DE FERNAN NUÑEZ ...,20004950,1,NUM,4,,"440706,59","4473659,52",99000065.0,MERCADO MUNICIPAL ANTON MARTIN,12.0,Mercado Municipal,PB,40,CARNICERIA M LOPEZ,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",472201,COMERCIO AL POR MENOR DE CARNICERIA,CARNICERIA M LOPEZ-DUQUE DE FERNAN NUÑEZ-4,290001655.0,10000226.0
4,10000238,1,CENTRO,102,EMBAJADORES,NaN,NaN,59,0,0,NaN,Agrupado,NaN,Abierto,253200,CALLE,DUQUE DE FERNAN NUÑEZ ...,20004950,1,NUM,4,,0,NaN,CALLE,DUQUE DE FERNAN NUÑEZ ...,20004950,1,NUM,4,,"440706,59","4473659,52",99000065.0,MERCADO MUNICIPAL ANTON MARTIN,12.0,Mercado Municipal,SO,13,CARNICERIA MARIANO ARAUJO,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",472201,COMERCIO AL POR MENOR DE CARNICERIA,CARNICERIA MARIANO ARAUJO-DUQUE DE FERNAN NUÑEZ-4,290001655.0,10000238.0


In [173]:
df_epi18_a.head()

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,id_local_norm,id_local_norm.values
0,10000116,1,CENTRO,105,UNIVERSIDAD,NaN,NaN,112,0,0,NaN,Agrupado,NaN,Abierto,524500,PLAZA,MOSTENSES ...,11006826,1,NUM,1,,0,NaN,PLAZA,MOSTENSES ...,11006826,1,NUM,1,,"439852,6","4474996,53",99000061.0,MERCADO MUNICIPAL LOS MOSTENSES,12.0,Mercado Municipal,1,17,PERFUMERIA DROGUERIA RICARDO / LOCAL 17 AL 22,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",477501,COMERCIO AL POR MENOR DE ARTICULOS DE PERFUMER...,PERFUMERIA DROGUERIA RICARDO / LOCAL 17 AL 22-...,290001655.0,NaN
1,10000162,1,CENTRO,104,JUSTICIA,NaN,NaN,88,0,0,NaN,Agrupado,NaN,Abierto,92900,CALLE,BARCELO ...,11005232,1,NUM,6,,0,NaN,CALLE,BARCELO ...,11005232,1,NUM,6,,"440692,59","4475369,53",99000056.0,MERCADO MUNICIPAL DE BARCELO,12.0,Mercado Municipal,0,101,POLLERIAS RODRIGUEZ & MARTINEZ,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",472206,"COMERCIO AL POR MENOR DE AVES, HUEVOS Y CAZA S...",POLLERIAS RODRIGUEZ & MARTINEZ-BARCELO-6,290001655.0,NaN
2,10000224,1,CENTRO,102,EMBAJADORES,NaN,NaN,35,0,0,NaN,Agrupado,NaN,Abierto,262000,CALLE,EMBAJADORES ...,11002143,1,NUM,41,,0,NaN,CALLE,EMBAJADORES ...,11002143,1,NUM,41,,"440295,59","4473246,53",99000057.0,MERCADO MUNICIPAL DE SAN FERNANDO,12.0,Mercado Municipal,PB,70,GARITO LOCAL 1 (PTO 70),G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",472407,ESTABLECIMIENTOS DE VENTA DE PLATOS PREPARADOS...,GARITO LOCAL 1 (PTO 70)-EMBAJADORES-41,290001655.0,NaN
3,10000226,1,CENTRO,102,EMBAJADORES,NaN,NaN,59,0,0,NaN,Agrupado,NaN,Abierto,253200,CALLE,DUQUE DE FERNAN NUÑEZ ...,20004950,1,NUM,4,,0,NaN,CALLE,DUQUE DE FERNAN NUÑEZ ...,20004950,1,NUM,4,,"440706,59","4473659,52",99000065.0,MERCADO MUNICIPAL ANTON MARTIN,12.0,Mercado Municipal,PB,40,CARNICERIA M LOPEZ,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",472201,COMERCIO AL POR MENOR DE CARNICERIA,CARNICERIA M LOPEZ-DUQUE DE FERNAN NUÑEZ-4,290001655.0,NaN
4,10000238,1,CENTRO,102,EMBAJADORES,NaN,NaN,59,0,0,NaN,Agrupado,NaN,Abierto,253200,CALLE,DUQUE DE FERNAN NUÑEZ ...,20004950,1,NUM,4,,0,NaN,CALLE,DUQUE DE FERNAN NUÑEZ ...,20004950,1,NUM,4,,"440706,59","4473659,52",99000065.0,MERCADO MUNICIPAL ANTON MARTIN,12.0,Mercado Municipal,SO,13,CARNICERIA MARIANO ARAUJO,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",472201,COMERCIO AL POR MENOR DE CARNICERIA,CARNICERIA MARIANO ARAUJO-DUQUE DE FERNAN NUÑEZ-4,290001655.0,NaN


In [163]:
df_epi19_a.reset_index(drop=True,inplace=True)

In [177]:
df_epi19_a.conc.values[1]

'POLLERIAS RODRIGUEZ & MARTINEZ-BARCELO-6'

In [176]:
df_epi18_a.conc.values[1]

'POLLERIAS RODRIGUEZ & MARTINEZ-BARCELO-6'

### El bueno

In [185]:
n = 0
for i in range(len(df_epi18_a.conc.values)):
    if df_epi18_a.conc.values[i] in df_epi19_a.conc.values:
        df_epi19_a.id_local_norm.values[i] = df_epi19_a.id_local.values[i]
        df_epi18_a.id_local_norm.values[i] = df_epi19_a.id_local_norm.values[i]
    n += 1
    print(df_epi18_a.conc.values[i],n)

PERFUMERIA DROGUERIA RICARDO / LOCAL 17 AL 22-MOSTENSES-1 1
POLLERIAS RODRIGUEZ & MARTINEZ-BARCELO-6 2
GARITO LOCAL 1 (PTO 70)-EMBAJADORES-41 3
CARNICERIA M LOPEZ-DUQUE DE FERNAN NUÑEZ-4 4
CARNICERIA MARIANO ARAUJO-DUQUE DE FERNAN NUÑEZ-4 5
MYA QUESOS ARTESANOS / PTOS 20 Y 21-SAN MIGUEL-11 6
PESCADERIA PEDRO CARBALLO-TOLEDO-109 7
WASHOKU PUESTOS 44 A 48-EMBAJADORES-41 8
CAFE AND TE (L-10)-ARENAL-9 9
AGUSTIN PEREZ PANIZO (CARNICERIA) / LOCAL 50-51-MOSTENSES-1 10
PESCADERIA FERNANDO-TOLEDO-109 11
PESCADERIA LA MADRILEÑA / LOCAL 68-69-MOSTENSES-1 12
ARTE FLORAL GALICIA ( L-10 )-MOSTENSES-1 13
RESTAURANTE MADRID FUSION-TIRSO DE MOLINA-15 14
JINGUANG LIU / LOCAL 90 AL 93-MOSTENSES-1 15
JINGUANG LIU / LOCAL 90 AL 93-MOSTENSES-1 16
CASA EMILIANO / LOCAL 4B AL 6B-MOSTENSES-1 17
CEREAL LOVER (PP, PTO.: 24-25)-DUQUE DE FERNAN NUÑEZ-4 18
ZONA 87-MOSTENSES-1 19
PESCADERIA JAVIER-BARCELO-6 20
AVES HUEVOS CAZA / LOCAL 52-53-MOSTENSES-1 21
PALACIO AVES HUEVOS CAZA / LOCAL 13 AL 16-MOSTENSES-1 22
POLL

AITA-GENERAL PERON-40 297
BEGOÑA BAR-BRAVO MURILLO-122 298
LA UVI  REPARACION DE CALZADO-SAN ENRIQUE-16 299
MARAVILLAS-BRAVO MURILLO-122 300
CHARCUTERIA JUAN / CJ 284-285-BRAVO MURILLO-122 301
EF CARNICERIA SALCHICHERIA / CJ 6 A 10-BRAVO MURILLO-122 302
EF CARNICERIA SALCHICHERIA / CJ 6 A 10-BRAVO MURILLO-122 303
EF CARNICERIA SALCHICHERIA / CJ 6 A 10-BRAVO MURILLO-122 304
CAFETERIA EXPRESATE-MARQUES DE VIANA-4 305
MARISCOS PESCADOS ANTONIO MUNICIO / CJ 254 A 255-BRAVO MURILLO-122 306
EL GOURMET DE JULIA / LOCAL 106-107-108-BRAVO MURILLO-122 307
RUIZ DE PASCUAL / CAJON 77-78-BRAVO MURILLO-122 308
DAYDAY/PTO31-33-GENERAL PERON-40 309
CLUB DE MALAS MADRES-GENERAL PERON-40 310
BIMBA & LOLA/PTO27-28-GENERAL PERON-40 311
MUSGO-GENERAL PERON-40 312
MANTEQUERIA LA VERDAD / CJ 69 A 71-BRAVO MURILLO-122 313
MANTEQUERIA LA VERDAD / CJ 69 A 71-BRAVO MURILLO-122 314
LA BOUTIQUE DE LA CARNE. CJ168-BRAVO MURILLO-122 315
CARNICERIA ISIDORO MARTIN GRAN SELECCION / CJ 224-225-BRAVO MURILLO-122 316
LA S

LECHERIA PTO. 8-BLAS CABRERA-125 540
CARNICERIA MADEX-MAQUEDA-117 541
CARNICERIA MADEX-MAQUEDA-117 542
PESCADERIA SANTIAGO-HIGUERAS-47 543
PESCADERIA-BLAS CABRERA-125 544
PESCADOS MUSANZ-GALICIA-3 545
LA FLOR DEL MAR-AGILA-1 546
PESCADERIA DOMINGO ALONSO-AGILA-1 547
PESCADERIA CADIERNO-AGILA-1 548
CARNICERIA LOS SEGOVIANOS-CAMARENA-235 549
HERMANOS BRAVO CARNICERIA-AGILA-1 550
CARNICERIA HERMANOS TAPIA CASQUERIA-GALICIA-3 551
PANADERIA BOLLERIA CROISSANTERIE-GALICIA-3 552
LACTEOS ROMERO (PUESTO 18 A 20)-LOS YEBENES-68 553
AVES HUEVOS CAZA-ILLESCAS-1 554
MARISCOS  VIZCAINO PESCADERIA-GALICIA-3 555
CARNES A. VIÑAMBRES-CEBREROS-19 556
CARNICERIA HERMANOS TAPIA CASQUERIA-GALICIA-3 557
PESCADOS Y MARISCOS MARIN-LOS YEBENES-68 558
JOSE LUIS-CEBREROS-19 559
POLLERIA HNOS REY-HIGUERAS-47 560
HUEVERIA-POLLERIA RAUL-BLAS CABRERA-125 561
AVES Y HUEVOS-VALLE INCLAN-24 562
ROTULO-VALLE INCLAN-24 563
FRUTAS Y VERDURAS VILLALBA (P30 Y P31)-HIGUERAS-47 564
CONGELADOS J. HERRERO-HIGUERAS-47 565
FRUTAS 

EUROTIENDAS-ALCALA-412 869
EUROTIENDAS-ALCALA-412 870
AVES HUEVOS Y CAZA-MARQUESA DE TORRECILLA-4 871
IS Y RA VARIANTES-ALCALA-412 872
ZARA HOME (PUESTOS 6 Y 7)-ARTURO SORIA-126 873
LOS 4 INGLETES-ARTURO SORIA-126 874
THE BODY SHOP-ARTURO SORIA-126 875
BIMBA Y LOLA (PUESTOS 36 Y 37)-ARTURO SORIA-126 876
LATERAL-ARTURO SORIA-126 877
ALIMENTACION ARMANDO-VIRGEN DEL VAL-9 878
LA FLOR DE ASCAO / L20 GUTIERRE DE CETINA 71-GUTIERRE DE CETINA-71 879
FRUTOS SECOS Y ESPECIAS-VIRGEN DE LA ALEGRIA-10 880
LIMIT 8-ALCALA-412 881
BEEP-ALCALA-412 882
EUFORIA-ALCALA-412 883
FRUTAS Y VERDURAS ABEJON-MARIA TERESA SAENZ DE HEREDIA-10 884
SEBAGO-ARTURO SORIA-126 885
TRES SOLES-ARTURO SORIA-126 886
MAGOVISION-ALCALA-412 887
MAGOVISION-ALCALA-412 888
RED PISO-CALERUEGA-17 889
PELUQUERIA MARIA-ALCALA-412 890
CHARCUTERIA RESTITUTO RICOTE / PUESTOS 59 Y 60-ALCALA-412 891
PASTELERIA CAFETERIA IPANEMA-ALCALA-412 892
PASTELERIA CAFETERIA IPANEMA-ALCALA-412 893
PESCADERIA Y MARISCOS DUFIPESCA SL-VIRGEN DE LA ALEGR

PRIMARK-ADOLFO BIOY CASARES-2 1181
JACK AND JONES / LOCAL 111 Y 112-ADOLFO BIOY CASARES-2 1182
ELENA HERNANDEZ-ADOLFO BIOY CASARES-2 1183
NEW YORKER-ADOLFO BIOY CASARES-2 1184
PRIMOR-SOSTRENEGRENES-ADOLFO BIOY CASARES-2 1185
PACO MARTINEZ-ADOLFO BIOY CASARES-2 1186
INSIDE-ADOLFO BIOY CASARES-2 1187
INTIMISSIMI-ADOLFO BIOY CASARES-2 1188
PIMKIE-ADOLFO BIOY CASARES-2 1189
STRADIVARIUS-ADOLFO BIOY CASARES-2 1190
G-STAR-ADOLFO BIOY CASARES-2 1191
SALSA-ADOLFO BIOY CASARES-2 1192
UNDIZ-ADOLFO BIOY CASARES-2 1193
SEPHORA-ADOLFO BIOY CASARES-2 1194
CLAIRE S-ADOLFO BIOY CASARES-2 1195
SPRINGFIELD-ADOLFO BIOY CASARES-2 1196
KIKO MAKE UP MILANO-ADOLFO BIOY CASARES-2 1197
PURIFICACION GARCIA-ADOLFO BIOY CASARES-2 1198
CAFETERIA RENFE-SIERRA DE GUADALUPE-10 1199
FNAC LA GAVIA-ADOLFO BIOY CASARES-2 1200
FNAC LA GAVIA-ADOLFO BIOY CASARES-2 1201
FNAC LA GAVIA-ADOLFO BIOY CASARES-2 1202
HUNKEMOLLER TIENDA-ADOLFO BIOY CASARES-2 1203
PARFOIS-ADOLFO BIOY CASARES-2 1204
PALPELO-ADOLFO BIOY CASARES-2 1205


...

RELAY-AGUSTIN DE FOXA-40 14265
JUGUETERIA POLY-AGUSTIN DE FOXA-40 14266
AVIS-AGUSTIN DE FOXA-40 14267
LA TIENDA 24 HORAS-AGUSTIN DE FOXA-40 14268
NAVELGAS-AGUSTIN DE FOXA-40 14269
LA ALACENA DE VICTOR MONTES-SAN MIGUEL-11 14270
EXARGIA / LOCALES 33 AL 35-EMBAJADORES-41 14271
J MARTIN IMPORTADOR DISTRIBUIDOR-ARENAL-9 14272
DIDECO-ARGANZUELA-22 14273
DIDECO-ARGANZUELA-22 14274
TERRA AUSTRALIS. NATURALEZA Y BIENESTAR-ARGANZUELA-22 14275
ASOC DE CREADORES DE MODA EN ESPAÑA-ARGANZUELA-22 14276
LOS PANCHITOS DE ENFRENTE/PTO 34-EMBAJADORES-41 14277
LOS PANCHITOS DE ENFRENTE/PTO 34-EMBAJADORES-41 14278
KOKOA-TIRSO DE MOLINA-15 14279
ORIGEN / (PLT. 1B) LOCAL 22 AL 29-MOSTENSES-1 14280
ORIGEN / (PLT. 1B) LOCAL 22 AL 29-MOSTENSES-1 14281
FRUTERIA / (PLT. 1A) LOCAL 7 A 10-MOSTENSES-1 14282
CONGELADOS LA MADRILEÑA/ LOCAL 85-86-MOSTENSES-1 14283
CONGELADOS LUCAS-DUQUE DE FERNAN NUÑEZ-4 14284
FRUTERIA FERNANDO-DUQUE DE FERNAN NUÑEZ-4 14285
BAR-DUQUE DE FERNAN NUÑEZ-4 14286
EL YANTAR DE AYER-SAN MIGUE

VIGARRI [F] [PUESTO 16 Y 18]-CUARENTA Y SEIS MERCAMADRID-4 14484
RAQUEL MARTINEZ YELO [F] [PUESTO 23 Y 25]-CUARENTA Y SEIS MERCAMADRID-4 14485
QUALIANZA [M0DULO 22]-LEGAZPI MERCAMADRID-9 14486
APERIDUL [M0DULO 30]-LEGAZPI MERCAMADRID-9 14487
ESENCIAS DEL MAR [M0DULO 36]-LEGAZPI MERCAMADRID-9 14488
IMPORTACO FOOD SERVICE [M0DULO 2-4]-LEGAZPI MERCAMADRID-9 14489
LUIS MARTIN ESQUIVIAS [PUESTO 126A]-PUERTA DE TOLEDO MERCAMADRID-2 14490
BRAMA AUSTRAL [PUESTO 16]-PUERTA DE TOLEDO MERCAMADRID-2 14491
VIESMAR [PUESTO 10A]-PUERTA DE TOLEDO MERCAMADRID-2 14492
FERJAVI [PUESTO 123]-PUERTA DE TOLEDO MERCAMADRID-2 14493
DISTRIBUCION DE MARISCOS RODRIGUEZ [PUESTO 124A]-PUERTA DE TOLEDO MERCAMADRID-2 14494
PEPRIMAR [PUESTO 95]-PUERTA DE TOLEDO MERCAMADRID-2 14495
CONFREMAR [PUESTO 92]-PUERTA DE TOLEDO MERCAMADRID-2 14496
[PUESTO 117] PUESTO DISPONIBLE-PUERTA DE TOLEDO MERCAMADRID-2 14497
CASADO PESCADOS Y MARISCOS [PUESTO 115]-PUERTA DE TOLEDO MERCAMADRID-2 14498
COOPERATIVA DE MAYORISTAS DE PESCADO 

PESCADOS Y MARISCOS VELASCO Y ESPINOS [PUESTO 21]-PUERTA DE TOLEDO MERCAMADRID-2 14721
FRIGAMADRID [PUESTO 19]-PUERTA DE TOLEDO MERCAMADRID-2 14722
[PUESTO 26]-PUERTA DE TOLEDO MERCAMADRID-2 14723
NOVAFICA [PUESTO 81]-PUERTA DE TOLEDO MERCAMADRID-2 14724
JAIME SORIANO [PUESTO 116]-PUERTA DE TOLEDO MERCAMADRID-2 14725
PESCADOS Y MARISCOS VELASCO Y ESPINOS [PUESTO 15]-PUERTA DE TOLEDO MERCAMADRID-2 14726
PESCADOS JOSE NAVARRO [PUESTO 133]-PUERTA DE TOLEDO MERCAMADRID-2 14727
DAORO [PUESTO 131]-PUERTA DE TOLEDO MERCAMADRID-2 14728
FERNANDO E HIJOS [PUESTO 102]-PUERTA DE TOLEDO MERCAMADRID-2 14729
PESCADOS UCEDO [PUESTO 94]-PUERTA DE TOLEDO MERCAMADRID-2 14730
PESCADOS UCEDO [PUESTO 94]-PUERTA DE TOLEDO MERCAMADRID-2 14731
[PUESTO 6]-PUERTA DE TOLEDO MERCAMADRID-2 14732
ISTOPESCA [PUESTO 120]-PUERTA DE TOLEDO MERCAMADRID-2 14733
[PUESTO 4A]-PUERTA DE TOLEDO MERCAMADRID-2 14734
ANGEL MOZOS RAMIREZ [PUESTO 99]-PUERTA DE TOLEDO MERCAMADRID-2 14735
CIAMAR [PUESTO 72]-PUERTA DE TOLEDO MERCAMADR

[M0DULO 16] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14953
[M0DULO 17]-LEGAZPI MERCAMADRID-9 14954
[M0DULO 18] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14955
[M0DULO 20] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14956
LECHE PASCUAL [M0DULO 22-24]-LEGAZPI MERCAMADRID-9 14957
[M0DULO 26]-LEGAZPI MERCAMADRID-9 14958
[M0DULO 29] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14959
[M0DULO 31]-LEGAZPI MERCAMADRID-9 14960
[M0DULO 34] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14961
[M0DULO 35 A] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14962
[M0DULO 37] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14963
[M0DULO 39] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14964
NO UTILIZAR [M0DULOS 40-42-44-46 Y 48]-LEGAZPI MERCAMADRID-9 14965
[M0DULO 40 A] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14966
[M0DULO 41] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14967
[M0DULO 42] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14968
[M0DULO 43] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14969
[M0DULO 44] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14970
[M0DULO 45] - NO UTILIZAR-LEGAZPI MERCAMADRID-9 14

[I] FRUSANGAR  [PUESTO 31]-CUARENTA Y OCHO MERCAMADRID-3 15093
[I] [PUESTO 32] - NO UTILIZAR-CUARENTA Y OCHO MERCAMADRID-3 15094
AJOS Y CEBOLLAS - EUROPATATAS [I] [PUESTO 33 OFICINAS]-CUARENTA Y OCHO MERCAMADRID-3 15095
LOCAL VACIO [PUESTO 34 ]-CUARENTA Y OCHO MERCAMADRID-3 15096
MANGOSTA TROPICAL [I] [PUESTO 35 CAMARA 1]-CUARENTA Y OCHO MERCAMADRID-3 15097
SALVADOR ORTIZ S.L.U.[I] [MODULO 35 CAMARA 4]-CUARENTA Y OCHO MERCAMADRID-3 15098
MANUEL ALONSO MARTIN[I] [PUESTO 35 CAMARA 2]-CUARENTA Y OCHO MERCAMADRID-3 15099
[I] [PUESTO 38] - NO UTILIZAR-CUARENTA Y OCHO MERCAMADRID-3 15100
[I] [PUESTO 39] - NO UTILIZAR-CUARENTA Y OCHO MERCAMADRID-3 15101
[I] [PUESTO 40] - NO UTILIZAR-CUARENTA Y OCHO MERCAMADRID-3 15102
[I] [PUESTO 41] - NO UTILIZAR-CUARENTA Y OCHO MERCAMADRID-3 15103
FELIX CATALAN [I] [PUESTO 42]-CUARENTA Y OCHO MERCAMADRID-3 15104
ENRIQUE Y FELIX DE DIEGO [I] [PUESTO 43]-CUARENTA Y OCHO MERCAMADRID-3 15105
PALACIOS ROCA [F3] [MODULO 1 A 6]-CINCUENTA MERCAMADRID-1 15106
[F3] [

In [186]:
df_epi19_a[df_epi19_a.conc == 'SIN ACTIVIDAD-ALCALA-412']

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,id_local_norm,id_local_norm.values
6296,270556930,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,0,0,NaN,Agrupado,NaN,Cerrado,88899208,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,SS,63,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,270556930.0,NaN
6297,270556931,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,0,0,NaN,Agrupado,NaN,Cerrado,88899208,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,15B,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,290001655.0,NaN
13950,280074159,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88899208,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,11,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280074159.0,NaN
13951,280074160,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88899208,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,12,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280074160.0,NaN
13988,280074197,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88899208,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,18,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280074197.0,NaN
13989,280074198,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88899208,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,21,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280074198.0,NaN
13990,280074199,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88899208,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,22,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280074199.0,NaN
14003,280074213,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88899208,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,40,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280074213.0,NaN
14016,280074232,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88899208,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA

In [187]:
df_epi18_a[df_epi18_a.conc == 'SIN ACTIVIDAD-ALCALA-412']

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,id_local_norm,id_local_norm.values
6316,270556930,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,0,0,NaN,Agrupado,NaN,Cerrado,88896929,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,SS,63,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,270556983.0,NaN
6317,270556931,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,0,0,NaN,Agrupado,NaN,Cerrado,88896929,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,15B,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,270556984.0,NaN
13717,280074159,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88896929,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,11,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280073922.0,NaN
13718,280074160,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88896929,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,12,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280073923.0,NaN
13754,280074196,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88896929,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,17,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280073959.0,NaN
13755,280074197,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88896929,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,18,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280073960.0,NaN
13756,280074198,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88896929,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,21,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280073961.0,NaN
13757,280074199,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88896929,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA NORTE,1.0,Centro Comercial,1,22,SIN ACTIVIDAD,NaN,NaN,NaN,NaN,NaN,NaN,SIN ACTIVIDAD-ALCALA-412,280073962.0,NaN
13769,280074213,15,CIUDAD LINEAL,1502,PUEBLO NUEVO,NaN,NaN,90,"445879,57","4476499,46",NaN,Agrupado,NaN,Cerrado,88896929,CALLE,ALCALA ...,20052956,1,NUM,412,,0,NaN,CALLE,ALCALA ...,20052956,1,NUM,412,,"445879,57","4476498,46",99000087.0,CENTRO COMERCIAL ALCALA

In [194]:
n = 0
for i in df_epi18_a.conc.values:
    if i in df_epi19_a.conc.values:
        df_epi19_a.loc[df_epi19_a.conc == i,'id_local_norm'] = max(df_epi19_a.loc[df_epi19_a.conc== i,'id_local'].values)
        df_epi18_a.loc[df_epi18_a.conc == i,'id_local_norm'] = df_epi19_a.loc[df_epi19_a.conc == i,'id_local_norm']
    n += 1
    print(i,n)

/home/dsc/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


PERFUMERIA DROGUERIA RICARDO / LOCAL 17 AL 22-MOSTENSES-1 1
POLLERIAS RODRIGUEZ & MARTINEZ-BARCELO-6 2
GARITO LOCAL 1 (PTO 70)-EMBAJADORES-41 3
CARNICERIA M LOPEZ-DUQUE DE FERNAN NUÑEZ-4 4
CARNICERIA MARIANO ARAUJO-DUQUE DE FERNAN NUÑEZ-4 5
MYA QUESOS ARTESANOS / PTOS 20 Y 21-SAN MIGUEL-11 6
PESCADERIA PEDRO CARBALLO-TOLEDO-109 7
WASHOKU PUESTOS 44 A 48-EMBAJADORES-41 8
CAFE AND TE (L-10)-ARENAL-9 9
AGUSTIN PEREZ PANIZO (CARNICERIA) / LOCAL 50-51-MOSTENSES-1 10
PESCADERIA FERNANDO-TOLEDO-109 11
PESCADERIA LA MADRILEÑA / LOCAL 68-69-MOSTENSES-1 12
ARTE FLORAL GALICIA ( L-10 )-MOSTENSES-1 13
RESTAURANTE MADRID FUSION-TIRSO DE MOLINA-15 14
JINGUANG LIU / LOCAL 90 AL 93-MOSTENSES-1 15
JINGUANG LIU / LOCAL 90 AL 93-MOSTENSES-1 16
CASA EMILIANO / LOCAL 4B AL 6B-MOSTENSES-1 17
CEREAL LOVER (PP, PTO.: 24-25)-DUQUE DE FERNAN NUÑEZ-4 18
ZONA 87-MOSTENSES-1 19
PESCADERIA JAVIER-BARCELO-6 20
AVES HUEVOS CAZA / LOCAL 52-53-MOSTENSES-1 21
PALACIO AVES HUEVOS CAZA / LOCAL 13 AL 16-MOSTENSES-1 22
POLL

OLIMPUS SPORT NUTRICION-AGUSTIN DE FOXA-40 209
OLIMPUS SPORT NUTRICION-AGUSTIN DE FOXA-40 210
ACCESSORIZE-AGUSTIN DE FOXA-40 211
EL OBRADOR DE ERNESTO PRIETO / BC 94-95-96-BOLIVIA-9 212
EL MERCADITO EL SABROSO-COLOMBIA-3 213
LA RETOUCHERIA DE MANUELA-CONCHA ESPINA-1 214
POLLERIA CARNICERIA / LOC. 1 Y 2-LOPEZ DE HOYOS-198 215
POLLERIA CARNICERIA / LOC. 1 Y 2-LOPEZ DE HOYOS-198 216
UCS NIÑOS EN ALGODON AUC-JUAN RAMON JIMENEZ-8 217
JUTECO / LC 16 Y 17-CONCHA ESPINA-1 218
P. POSTIGO ESPECIALIDADES / BC 56 A 60-BOLIVIA-9 219
SU FRUTA (EL SABROSO)-SUERO DE QUIÑONES-4 220
CARNICERIA-SUERO DE QUIÑONES-4 221
BELROS-CONCHA ESPINA-1 222
BELROS-CONCHA ESPINA-1 223
RAZA NOSTRA / LOC.34 AL 38-BOLIVIA-9 224
GEOX-CONCHA ESPINA-1 225
PESCADERIA-NIEREMBERG-5 226
SILOMM-CONCHA ESPINA-1 227
AN SPORT-CONCHA ESPINA-1 228
LA BODEGA JESUS LORENZO-SAN ENRIQUE-16 229
AHM PINCHOS Y MERCADOS SC-SAN ENRIQUE-16 230
CARNICERIA SALCHICHERIA / CJ 119-120-BRAVO MURILLO-122 231
CARNICERIA SALCHICHERIA / CJ 119-120-BRAVO

EL CARMEN-CASTELLANA-261 393
CARNICERIA LOS SERRANOS-MONFORTE DE LEMOS-36 394
VILLA GRANADOS-MONFORTE DE LEMOS-36 395
RESTAURANTE DOMIN0-ISABEL COLBRAND-10 396
PESCADERIA T. BURGOS-MONFORTE DE LEMOS-36 397
LASHES & GO-NURIA-59 398
LASHES & GO-NURIA-59 399
LEFTIES. LOCAL 007/010-MONFORTE DE LEMOS-36 400
FLORISTERIA-NURIA-59 401
LA PLAZA DIA-CERRO DE LA CARRASQUETA-134 402
LA PLAZA DIA-CERRO DE LA CARRASQUETA-134 403
LA PLAZA DIA-CERRO DE LA CARRASQUETA-134 404
LA PLAZA DIA-CERRO DE LA CARRASQUETA-134 405
LA PLAZA DIA-CERRO DE LA CARRASQUETA-134 406
LA PLAZA DIA-CERRO DE LA CARRASQUETA-134 407
LA PLAZA DIA-CERRO DE LA CARRASQUETA-134 408
LA PLAZA DIA-CERRO DE LA CARRASQUETA-134 409
CENTRO DE ESTETICA  GUADALUPE CARRASCO-LEOPOLDO ALAS CLARIN-8 410
CENTRO DE ESTETICA  GUADALUPE CARRASCO-LEOPOLDO ALAS CLARIN-8 411
UNICO-MONFORTE DE LEMOS-36 412
UNICO-MONFORTE DE LEMOS-36 413
UNICO-MONFORTE DE LEMOS-36 414
MARCO ALDANY-MONFORTE DE LEMOS-36 415
MARCO ALDANY-MONFORTE DE LEMOS-36 416
LA MAFIA-M

ODONTOLOGIA POR ESPECIALIDAD (L-150)-OLIVA DE PLASENCIA-1 606
CORSETERIA ROSI LENCERIA (L-480)-OLIVA DE PLASENCIA-1 607
DREAMFIT-POBLADOS-58 608
HERMANOS ESCORIAL-SAN PATRICIO-1 609
POLLERIA PACO-LAGUNA-107 610
POLLERIA HUEVERIA-ANTONIO LOPEZ-6 611
POLLERIA HUEVERIA-ANTONIO LOPEZ-6 612
JUAN CARLOS AVES-CAZA-HUEVOS-SAN PATRICIO-1 613
JOYERIA ESCUDERO-ANTONIO LOPEZ-6 614
BAR/PTO 10 Y 11-ANTONIO LOPEZ-47 615
POLLERIA MAMEL-NUESTRA SEÑORA DE FATIMA-19 616
PESCADOS Y MARISCOS HNOS. SANTIAGO BOLAÑOS,-LAGUNA-107 617
GARCIA PEDRAZUELA MANUEL-LAGUNA-107 618
PESCADERIA-CAMINO VIEJO DE LEGANES-177 619
GRAN PESCADERIA MARISQUERIA-ANTONIO LOPEZ-6 620
CARNICERIA NICOLAS Y ANGEL VALLINOT-MATILDE HERNANDEZ-100 621
PESCADOS SANTIAGO-ANTONIO LOPEZ-6 622
PESCADOS Y MARISCOS-ANTONIO LOPEZ-47 623
CARNICERIA HERMANOS MOLERO-NUESTRA SEÑORA DE FATIMA-19 624
CARNICERIA AGUSTIN-NUESTRA SEÑORA DE FATIMA-19 625
CARNICERIA-NUESTRA SEÑORA DE FATIMA-19 626
CARNICERIA MAXI-LAGUNA-107 627
ROPA Y COMPLEMENTOS-ANTONIO L

...

FRUTAS OLIVAR [B] [PUESTOS 19 A 27]-CUARENTA Y DOS MERCAMADRID-1 14586
FRUTAS E. SANCHEZ [B] [PUESTOS 36-38]-CUARENTA Y DOS MERCAMADRID-1 14587
JOSE MARIA NADADOR E HIJO [B] [PUESTOS 45 Y 47]-CUARENTA Y DOS MERCAMADRID-1 14588
AGUSTIN BRAVO [B] [PUESTO 42]-CUARENTA Y DOS MERCAMADRID-1 14589
HERMANOS FERNANDEZ L0PEZ [B] [PUESTOS 49 A 57]-CUARENTA Y DOS MERCAMADRID-1 14590
COMERCIAL PATALETA [B] [PUESTOS 50 A 54]-CUARENTA Y DOS MERCAMADRID-1 14591
FRUTAS HEREDIA [B] [PUESTOS 46 Y 48]-CUARENTA Y DOS MERCAMADRID-1 14592
HERMANOS FERNANDEZ LOPEZ [B] [PUESTOS 51A 57]-CUARENTA Y DOS MERCAMADRID-1 14593
CARMEN MARTINEZ ORTEGA [B] [PUESTO 44]-CUARENTA Y DOS MERCAMADRID-1 14594
NUFRI [B] [PUESTOS 29 A 35]-CUARENTA Y DOS MERCAMADRID-1 14595
FRUTAS SANTI [B] [PUESTO 24]-CUARENTA Y DOS MERCAMADRID-1 14596
FRUTAS CANDIL [C] [PUESTO 33 A 41]-CUARENTA Y CUATRO MERCAMADRID-4 14597
RIAZA AZAIR [C] [PUESTO 31]-CUARENTA Y CUATRO MERCAMADRID-4 14598
FRUTAS TRUEBA [C] [PUESTO 22 Y 24]-CUARENTA Y CUATRO MERC

MEDINA 3 [M0DULO 32-34]-LEGAZPI MERCAMADRID-9 14707
DISTRIBUCIONES AVICOLAS FRIGORIFICAS [M0DULO 38]-LEGAZPI MERCAMADRID-9 14708
CERRADO [MODULO 21]-LEGAZPI MERCAMADRID-9 14709
ITALICA IMPORTACIONES EXPORTACIONES [M0DULO 23]-LEGAZPI MERCAMADRID-9 14710
ENVASES Y EMBALAJES MUGO [M0DULO 28]-LEGAZPI MERCAMADRID-9 14711
CHICKENGAR [M0DULO 33]-LEGAZPI MERCAMADRID-9 14712
EL CEBADERO [M0DULOS 6-8-10-12-14-16] - ANTES DIOCARNES-LEGAZPI MERCAMADRID-9 14713
EL CEBADERO [M0DULOS 6-8-10-12-14-16] - ANTES DIOCARNES-LEGAZPI MERCAMADRID-9 14714
CENTRAL DE CARNES MADRID NORTE [M0DULOS 35-37-39-41-43-45-47]-LEGAZPI MERCAMADRID-9 14715
CENTRAL DE CARNES MADRID NORTE [M0DULOS 35-37-39-41-43-45-47]-LEGAZPI MERCAMADRID-9 14716
MEDINA 3 [M0DULO 1- 3-5-7-9-11-13]-LEGAZPI MERCAMADRID-9 14717
[M0DULOS 27-29]-LEGAZPI MERCAMADRID-9 14718
[M0DULOS 27-29]-LEGAZPI MERCAMADRID-9 14719
MARE NOSTRUM [PUESTO 74]-PUERTA DE TOLEDO MERCAMADRID-2 14720
PESCADOS Y MARISCOS VELASCO Y ESPINOS [PUESTO 21]-PUERTA DE TOLEDO MER

AHORRAMAS MERCAMADRID [C1A][EJE 2/7]-PUERTA DE TOLEDO MERCAMADRID-3 14833
O'CAMPO GONZALEZ [C1A][2]-PUERTA DE TOLEDO MERCAMADRID-3 14834
PUERTO DE PALOS [C1A][3]-PUERTA DE TOLEDO MERCAMADRID-3 14835
CONGELADOS CIENTOCINCO [C1A][4]-PUERTA DE TOLEDO MERCAMADRID-3 14836
ORTEGA 58[C1A][5]-PUERTA DE TOLEDO MERCAMADRID-3 14837
AHORRAMAS MERCAMADRID [C1A][6]-PUERTA DE TOLEDO MERCAMADRID-3 14838
AHORRAMAS MERCAMADRID [C1A][6]-PUERTA DE TOLEDO MERCAMADRID-3 14839
ACTIVIDADES DE USUARIOS EN ZONAS COMUNES-MADRID MERCAMADRID-3 14840
SIN ACTIVIDAD-CUARENTA Y OCHO MERCAMADRID-3 14841
SIN ACTIVIDAD-CUARENTA Y OCHO MERCAMADRID-3 14842
SIN ACTIVIDAD-CUARENTA Y OCHO MERCAMADRID-3 14843
SIN ACTIVIDAD-CUARENTA Y OCHO MERCAMADRID-3 14844
SIN ACTIVIDAD-CUARENTA Y OCHO MERCAMADRID-3 14845
SIN ACTIVIDAD-CUARENTA Y OCHO MERCAMADRID-3 14846
SIN ACTIVIDAD-CUARENTA Y OCHO MERCAMADRID-3 14847
SIN ACTIVIDAD-CUARENTA Y OCHO MERCAMADRID-3 14848
SIN ACTIVIDAD-CUARENTA Y OCHO MERCAMADRID-3 14849
SIN ACTIVIDAD-CUARENTA 

PRODUCTOS ALIMENTICIOS LAYMAR [PUESTO 76B]-PUERTA DE TOLEDO MERCAMADRID-2 14988
SANYAMAR [PUESTO 93]-PUERTA DE TOLEDO MERCAMADRID-2 14989
CERRADO [PUESTO 93B]-PUERTA DE TOLEDO MERCAMADRID-2 14990
[PUESTO 103] UNIFICADO CON 290001413-PUERTA DE TOLEDO MERCAMADRID-2 14991
[PUESTO 122] - UNIFICADO CON 290001180-PUERTA DE TOLEDO MERCAMADRID-2 14992
ISTOPESCA [PUESTO 122]-PUERTA DE TOLEDO MERCAMADRID-2 14993
[PUESTO 127] PUESTO DISPONIBLE-PUERTA DE TOLEDO MERCAMADRID-2 14994
JULIO PEREZ [PUESTO 128]-PUERTA DE TOLEDO MERCAMADRID-2 14995
[PUESTO 128] - NO UTILIZAR-PUERTA DE TOLEDO MERCAMADRID-2 14996
SANJUMAR [PUESTO 143A]-PUERTA DE TOLEDO MERCAMADRID-2 14997
[PUESTO 149]-PUERTA DE TOLEDO MERCAMADRID-2 14998
PESCA XXI [PUESTO 155]-PUERTA DE TOLEDO MERCAMADRID-2 14999
MARIÑA FISH [PUESTO 156A]-PUERTA DE TOLEDO MERCAMADRID-2 15000
MOLIPESCA [PUESTO 156B]-PUERTA DE TOLEDO MERCAMADRID-2 15001
EL SEDAL DE MERCAMADRID-PUERTA DE TOLEDO MERCAMADRID-2 15002
ADEPESCA-PUERTA DE TOLEDO MERCAMADRID-2 15003

FRUTALITY, S.L. [F4] [MODULO 2]-CINCUENTA MERCAMADRID-3 15119
[F4] [MODULO 3] - NO UTILIZAR-CINCUENTA MERCAMADRID-3 15120
[F4] [MODULO 4] - NO UTILIZAR-CINCUENTA MERCAMADRID-3 15121
[F4] [MODULO 5] - NO UTILIZAR-CINCUENTA MERCAMADRID-3 15122
[F4] [MODULO 6] - NO UTILIZAR-CINCUENTA MERCAMADRID-3 15123
[F4] [MODULO 7] - NO UTILIZAR-CINCUENTA MERCAMADRID-3 15124
[F4] [MODULO 8] - NO UTILIZAR-CINCUENTA MERCAMADRID-3 15125
FRUTAS A-Z [F4] [MODULO 9]-CINCUENTA MERCAMADRID-3 15126
[F4] [MODULO 10] - NO UTILIZAR-CINCUENTA MERCAMADRID-3 15127
MERCABANGLA [F4] [MODULO 11]-CINCUENTA MERCAMADRID-3 15128
[F4] [MODULO 12] - NO UTILIZAR-CINCUENTA MERCAMADRID-3 15129
PALACIOS ROCA [F4] [MODULO 13]-CINCUENTA MERCAMADRID-3 15130
COMFRESH IBERIAN [F4] [MODULO 14]-CINCUENTA MERCAMADRID-3 15131
AHORRAMAS MERCAMADRID [G1][A]-CUARENTA Y UNO MERCAMADRID-1 15132
NUFRI [G1] [B]-CUARENTA Y UNO MERCAMADRID-1 15133
SOCIEDAD ESTATAL DE MERCADOS CENTRALES DE ABASTECIMIENTO [G2]-CUARENTA Y UNO MERCAMADRID-7 15134
HIP

In [193]:
df_epi18_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15234 entries, 0 to 15233
Data columns (total 48 columns):
id_local                     15234 non-null int64
id_distrito_local            15234 non-null int64
desc_distrito_local          15234 non-null object
id_barrio_local              15234 non-null int64
desc_barrio_local            15234 non-null object
cod_barrio_local             0 non-null float64
id_seccion_censal_local      0 non-null float64
desc_seccion_censal_local    15234 non-null int64
coordenada_x_local           15234 non-null object
coordenada_y_local           15234 non-null object
id_tipo_acceso_local         0 non-null float64
desc_tipo_acceso_local       15234 non-null object
id_situacion_local           0 non-null float64
desc_situacion_local         15234 non-null object
id_vial_edificio             15234 non-null int64
clase_vial_edificio          15234 non-null object
desc_vial_edificio           15234 non-null object
id_ndp_edificio              15234 non-nu

In [195]:
df_epi18_a.to_csv('epi_18_a')
df_epi19_a.to_csv('epi_19_a')

In [190]:
df_epi18_a.drop(columns='id_local_norm.values',inplace=True)

/home/dsc/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [191]:
df_epi19_a.drop(columns='id_local_norm.values',inplace=True)

In [78]:
df_epi19_a['conc'].values[0:10]

array(['FOTO VALVANERA-NUESTRA SEÑORA DE VALVANERA-97',
       'BANCO SANTANDER-CAMPIÑA-5', 'SIN ACTIVIDAD-MANUEL MAROTO-47',
       'SIN ACTIVIDAD-ALBUFERA-46', 'SIN ACTIVIDAD-ALBUFERA-46',
       'PESCADERIA-ANDALUCES DEL POZO-9', 'SIN ACTIVIDAD-SERENA-270',
       'SIN ACTIVIDAD-SERENA-270', 'SIN ACTIVIDAD-SERENA-270',
       'SIN ACTIVIDAD-SERENA-270'], dtype=object)

In [80]:
df_epi18_a[df_epi18_a.conc == 'FOTO VALVANERA-NUESTRA SEÑORA DE VALVANERA-97']

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,id_local_norm
145527,280053954,11,CARABANCHEL,1104,VISTA ALEGRE,NaN,NaN,78,0,0,NaN,Agrupado,NaN,Abierto,540100,AVENIDA,NUESTRA SEÑORA DE VALVANERA ...,31054600,1,NUM,97,B,0,NaN,AVENIDA,NUESTRA SEÑORA DE VALVANERA ...,31054600,1,NUM,97,B,"437349,6","4471455,54",99000035.0,GALERIA DE ALIMENTACION VALVANERA,4.0,Galeria de Alimentacion,PB,EX1,FOTO VALVANERA,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",477805,COMERCIO AL POR MENOR DE MATERIAL FOTOGRAFICO ...,FOTO VALVANERA-NUESTRA SEÑORA DE VALVANERA-97,280053954.0


In [63]:
%%timeit
df_epi18_a.loc[df_epi18_a.conc == 'MULTIOPTICAS-CALDERILLA-1','id_local_norm'] = df_epi19_a.loc[df_epi19_a.conc == 'MULTIOPTICAS-CALDERILLA-1','id_local'].values[0]

48.4 ms ± 3.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
conc_un_a[0]

'FOTO VALVANERA-NUESTRA SEÑORA DE VALVANERA-97'

In [60]:
df_epi19_a[df_epi19_a.conc == 'FOTO VALVANERA-NUESTRA SEÑORA DE VALVANERA-97']

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,id_local_norm
252,280053954,11,CARABANCHEL,1104,VISTA ALEGRE,NaN,NaN,78,0,0,NaN,Agrupado,NaN,Abierto,540100,AVENIDA,NUESTRA SEÑORA DE VALVANERA ...,31054600,1,NUM,97,B,0,NaN,AVENIDA,NUESTRA SEÑORA DE VALVANERA ...,31054600,1,NUM,97,B,"437349,6","4471455,54",99000035.0,GALERIA DE ALIMENTACION VALVANERA,4.0,Galeria de Alimentacion,PB,EX1,FOTO VALVANERA,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",477805,COMERCIO AL POR MENOR DE MATERIAL FOTOGRAFICO ...,FOTO VALVANERA-NUESTRA SEÑORA DE VALVANERA-97,99000035.0


In [61]:
df_epi18_a[df_epi18_a.conc == 'FOTO VALVANERA-NUESTRA SEÑORA DE VALVANERA-97']

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,id_local_norm
145527,280053954,11,CARABANCHEL,1104,VISTA ALEGRE,NaN,NaN,78,0,0,NaN,Agrupado,NaN,Abierto,540100,AVENIDA,NUESTRA SEÑORA DE VALVANERA ...,31054600,1,NUM,97,B,0,NaN,AVENIDA,NUESTRA SEÑORA DE VALVANERA ...,31054600,1,NUM,97,B,"437349,6","4471455,54",99000035.0,GALERIA DE ALIMENTACION VALVANERA,4.0,Galeria de Alimentacion,PB,EX1,FOTO VALVANERA,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",477805,COMERCIO AL POR MENOR DE MATERIAL FOTOGRAFICO ...,FOTO VALVANERA-NUESTRA SEÑORA DE VALVANERA-97,280053954.0


In [56]:
df_epi19_a[df_epi19_a.conc == 'MULTIOPTICAS-CALDERILLA-1']

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,id_local_norm
151861,280052463,11,CARABANCHEL,1106,BUENAVISTA,NaN,NaN,198,"437367,57","4468536,54",NaN,Agrupado,NaN,Abierto,31000314,CALLE,CALDERILLA ...,31025055,1,NUM,1,,0,NaN,CALLE,CALDERILLA ...,31025055,1,NUM,1,,"437367,57","4468536,54",99000252.0,CENTRO COMERCIAL ISLAZUL,1.0,Centro Comercial,PB,67E,MULTIOPTICAS,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",477806,COMERCIO AL POR MENOR DE MATERIAL DE OPTICA,MULTIOPTICAS-CALDERILLA-1,280052463.0


In [64]:
df_epi18_a[df_epi18_a.conc == 'MULTIOPTICAS-CALDERILLA-1']

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,id_situacion_local,desc_situacion_local,id_vial_edificio,clase_vial_edificio,desc_vial_edificio,id_ndp_edificio,id_clase_ndp_edificio,nom_edificio,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,nom_acceso,num_acceso,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,conc,id_local_norm
76240,110003820,11,CARABANCHEL,1106,BUENAVISTA,NaN,NaN,198,"437367,57","4468536,54",NaN,Agrupado,NaN,Abierto,31000314,CALLE,CALDERILLA ...,31025055,1,NUM,1,,0,NaN,CALLE,CALDERILLA ...,31025055,1,NUM,1,,"437367,57","4468536,54",99000252.0,CENTRO COMERCIAL ISLAZUL,1.0,Centro Comercial,PB,35,MULTIOPTICAS,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHICULOS DE...",477806,COMERCIO AL POR MENOR DE MATERIAL DE OPTICA,MULTIOPTICAS-CALDERILLA-1,280052463.0


In [216]:
df_epi19.groupby(['id_local']).count().values > 1

array([[ True,  True,  True, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [22]:
df_epi19.id_epigrafe.fillna(999, inplace=True)

In [23]:
np.unique(df_epi19.id_epigrafe.values.astype(int))

ValueError: invalid literal for int() with base 10: 'PTECO1'

Voy a eliminar las secciones de los epigrafes que no me interesan: 
id_seccion_drop = ['A', 'B', 'C', 'D'. 'E', 'F', 'H', 'K', 'N', 'O', 'Q', 'U']. Los Z, ver si se me escapa algún null que tenga rotulo
id_division_drop = [46, 55, 58,59,60,90,91,94]
id_epigrafe_drop = [451001,451002,452004,681001,682001,851001,851002,851007,851008,851009,852001,852002,852003,852004,853001,853002,853003,854001,854002,854003]


n.c.o.p. = no comprendidos en otras partes

Los Z, ver si se me escapa algún null que tenga rotulo



A AGRICULTURA, GANADERÍA, SILVICULTURA Y PESCA ... -> filtrar

B INDUSTRIAS EXTRACTIVAS -> filtrar

C INDUSTRIA MANUFACTURERA ¿? -> filtrar

D SUMINISTRO DE ENERGÍA ELÉCTRICA, GAS, VAPOR Y AIRE ACONDICIONADO -> filtrar

E SUMINISTRO DE AGUA, ACTIVIDADES DE SANEAMIENTO, GESTIÓN DE RESIDUOS Y DESCONTAMINACIÓN -> filtrar

F CONSTRUCCIÓN -> filtrar

G COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACIÓN DE VEHÍCULOS 
DE MOTOR Y MOTOCICLETAS 

46 C OMERCIO AL POR MAYOR E INTERMEDIARIOS DEL COMERCIO , EXCEPTO DE VEHÍCULOS DE MOTOR Y
MOTOCICLETAS -> filtrar

47 C OMERCIO AL POR MENOR , EXCEPTO DE VEHÍCULOS DE MOTOR Y MOTOCICLETAS
Me quedo con los epigrafes que varian por encima de la media tanto en creación como en destrucción

H TRANSPORTE Y ALMACENAMIENTO -> filtrar. Necesitas mucha infra y es para pequeños negocios. Quizá analizar:
53 S ERVICIOS DE MENSAJERÍA , ACTIVIDADES POSTALES Y DE CORREO

I HOSTELERÍA -> este va a ser un melon... Pensar que hacer

J. INFORMACION Y COMUNICACIONES -> filtrar. Analizar si cambia con foco en 610002 Locutorios

K ACTIVIDADES FINANCIERAS Y DE SEGUROS -> filtrar

L ACTIVIDADES INMOBILIARIAS -> revisar si esto varía mucho. si no, filtrar

M ACTIVIDADES PROFESIONALES, CIENTÍFICAS Y TÉCNICAS -> analizar si hay mucho cambio pero en princio, filtrar
Quizá analizar evolución de:
75 A CTIVIDADES VETERINARIAS

N ACTIVIDADES ADMINISTRATIVAS Y SERVICIOS AUXILIARES -> filtrar

O ADMINISTRACIÓN PÚBLICA Y DEFENSA; SEGURIDAD SOCIAL OBLIGATORIA -> filtrar

P EDUCACION -> filtrar todo salvo:
851003 Centros de Cuidado infantil
855001 Autoescuela
855002 Enseñanza no reglada (deportiva y recreativa, cultural, clases de recuperación,
informática) n.c.o.p.
855003 Enseñanza de idiomas

Q ACTIVIDADES SANITARIAS Y DE SERVICIOS SOCIALES -> Filtrar

R ACTIVIDADES ARTÍSTICAS, RECREATIVAS Y DE ENTRETENIMIENTO
90 A CTIVIDADES DE CREACIÓN , ARTÍSTICAS Y ESPECTÁCULOS
91 A CTIVIDADES DE BIBLIOTECAS , ARCHIVOS , MUSEOS Y OTRAS ACTIVIDADES CULTURALES .
92 A CTIVIDADES DE JUEGOS DE AZAR Y APUESTAS
920002 Juegos de azar y apuestas de gestión privada (Bingos, casinos, máquinas tragaperras) .
93 ACTIVIDADES DEPORTIVAS

S OTROS SERVICIOS
94 - Otros
95 R EPARACIÓN DE ORDENADORES , EFECTOS PERSONALES Y ARTÍCULOS DE USO DOMÉSTICO - Dejar
96 O TROS SERVICIOS PERSONALES

U ACTIVIDADES DE ORGANIZACIONES Y ORGANISMOS EXTRATERRITORIALES
99 A CTIVIDADES DE ORGANIZACIONES Y ORGANISMOS EXTRATERRITORIALES

Z LOCAL SIN ACTIVIDAD
000000 L OCAL SIN A CTIVIDAD


In [8]:
df_epi19.columns

Index(['id_local', 'id_distrito_local', 'desc_distrito_local',
       'id_barrio_local', 'desc_barrio_local', 'cod_barrio_local',
       'id_seccion_censal_local', 'desc_seccion_censal_local',
       'coordenada_x_local', 'coordenada_y_local', 'id_tipo_acceso_local',
       'desc_tipo_acceso_local', 'id_situacion_local', 'desc_situacion_local',
       'id_vial_edificio', 'clase_vial_edificio', 'desc_vial_edificio',
       'id_ndp_edificio', 'id_clase_ndp_edificio', 'nom_edificio',
       'num_edificio', 'cal_edificio', 'secuencial_local_PC', 'id_vial_acceso',
       'clase_vial_acceso', 'desc_vial_acceso', 'id_ndp_acceso',
       'id_clase_ndp_acceso', 'nom_acceso', 'num_acceso', 'cal_acceso',
       'coordenada_x_agrupacion', 'coordenada_y_agrup', 'id_agrupacion',
       'nombre_agrupacion', 'id_tipo_agrup', 'desc_tipo_agrup',
       'id_planta_agrupado', 'id_local_agrupado', 'rotulo', 'id_seccion',
       'desc_seccion', 'id_division', 'desc_division', 'id_epigrafe',
       'desc_epi

In [ ]:
df_epi19[df_epi19.]

In [9]:
df_epi19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163316 entries, 0 to 163315
Data columns (total 48 columns):
id_local                     163316 non-null int64
id_distrito_local            163316 non-null int64
desc_distrito_local          163316 non-null object
id_barrio_local              163316 non-null int64
desc_barrio_local            163316 non-null object
cod_barrio_local             0 non-null float64
id_seccion_censal_local      0 non-null float64
desc_seccion_censal_local    163316 non-null int64
coordenada_x_local           163316 non-null object
coordenada_y_local           163316 non-null object
id_tipo_acceso_local         0 non-null float64
desc_tipo_acceso_local       163316 non-null object
id_situacion_local           0 non-null float64
desc_situacion_local         163316 non-null object
id_vial_edificio             163316 non-null int64
clase_vial_edificio          163316 non-null object
desc_vial_edificio           163316 non-null object
id_ndp_edificio           

In [22]:
df_prueba = df_epi19.copy()

In [23]:
df_prueba.drop(['cod_barrio_local',
                'id_seccion_censal_local',
               'id_tipo_acceso_local',
               'id_situacion_local',
               'id_vial_acceso','coordenada_x_agrupacion', 'coordenada_y_agrup', 'id_agrupacion',
       'nombre_agrupacion', 'id_tipo_agrup', 'desc_tipo_agrup',
       'id_planta_agrupado', 'id_local_agrupado'], axis = 1, inplace = True)

In [24]:
df_prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163316 entries, 0 to 163315
Data columns (total 35 columns):
id_local                     163316 non-null int64
id_distrito_local            163316 non-null int64
desc_distrito_local          163316 non-null object
id_barrio_local              163316 non-null int64
desc_barrio_local            163316 non-null object
desc_seccion_censal_local    163316 non-null int64
coordenada_x_local           163316 non-null object
coordenada_y_local           163316 non-null object
desc_tipo_acceso_local       163316 non-null object
desc_situacion_local         163316 non-null object
id_vial_edificio             163316 non-null int64
clase_vial_edificio          163316 non-null object
desc_vial_edificio           163316 non-null object
id_ndp_edificio              163316 non-null int64
id_clase_ndp_edificio        163316 non-null int64
nom_edificio                 163316 non-null object
num_edificio                 163316 non-null int64
cal_edificio 

In [25]:
df_prueba.dropna(inplace = True)

In [26]:
len(df_epi19), len(df_prueba)

(163316, 121892)